#### Dependencies

In [1]:
# Pickle file dependencies
from tacc_stats.pickler.job_stats import Job
import cPickle as pickle

In [2]:
# System dependencies
from os import listdir
import time as clock
from IPython.display import clear_output

In [26]:
# Data manipulation dependencies
import pandas as pd
import numpy as np
from numpy import diff, amax, zeros, maximum, mean, isnan, trapz

In [4]:
# Directory of all pickled jobs via comet
# new_dir = '/oasis/projects/nsf/sys200/stats/xsede_stats/archive'
source_dir = '/oasis/projects/nsf/sys200/tcooper/xsede_stats/comet_pickles/'

# Directory of pre-cleaned job files
dates_dir = './modules/data/dates(2016)/'

# Job ids we are looking for
goals = [ '4721174', '4726104', '4678353' ]

# Directory to save to
save_dir = './'

In [5]:
dates_dict = {date:dates_dir+date for date in listdir(dates_dir)}
dates_list = dates_dict.keys()

In [6]:
len(dates_list)

97

#### Read in jobs from cleaned jobs directory

In [7]:
def prep_target( date_file ):
    jobs_list = []
    
    # open file and read the content in a list
    with open(date_file, 'r') as f:
        lines = f.readlines()
    
        for jobid in lines:
            current = jobid[:-1]
            jobs_list.append(current)
    
    return jobs_list

# Access and open pickled job files
**Process:**
    - Iterate through the non-empty date folders available in source_dir
    - A file is saved in valid_jobs if:
        * The pickled file is a Job object
        * The job ran for more than 6 cycles (1 hour)
        * The total number of jobs saved at the end of the previous date folder is less than 1000
            _This is purely to keep the computations manageable according to compute time requested_
    - Exceptions are skipped

In [11]:
job_objects = []
t0 = clock.time()
total = 0

for i in range(len(dates_list)):
    target_date = dates_list[i]
    target_file = dates_dict[ target_date ]
    jobids = prep_target( target_file )
    n = len(jobids)

    for jobid in jobids:
        total += 1
        clear_output(wait=True)
        print("Processing file {} of {} files \t ({}% of total files)".format(total, n, np.round( total/n*100, 2)))
        
        pickle_file = open( source_dir+target_date+'/'+jobid, 'rb')
        job_file = pickle.load(pickle_file)
        if job_file.id in goals:
            job_objects.append(job_file)
        pickle_file.close()      
            
        t2 = clock.time()
        print
        print("Run time: {}s".format(np.round(t2-t0, 1)))

Processing file 37409 of 1513 files 	 (2400% of total files)

Run time: 2321.1s


In [52]:
for job in job_objects:
    print(job.id)
    print(job)
    print

4678353

4726104

4721174



### Utils prepping

In [19]:
class utils():
    def __init__(self, job):
        freq_list = {"intel_snb" : 2.7, "intel_ivb" : 2.8, "intel_hsw" : 2.3,
                     "intel_bdw" : 2.6, "intel_knl" : 1.4, "intel_skx" : 2.1}
        imc_list  = ["intel_snb_imc", "intel_ivb_imc", "intel_hsw_imc",
                     "intel_bdw_imc", "intel_knl_mc_dclk", "intel_skx_imc"]
        cha_list = ["intel_knl_cha", "intel_skx_cha"]
        self.job = job
        self.nhosts = len(job.hosts.keys())
        self.hostnames  = sorted(job.hosts.keys())
        self.wayness = int(job.acct['cores'])/int(job.acct['nodes'])
        self.hours = ((job.times[:] - job.times[0])/3600.).astype(float)
        self.t = job.times
        self.nt = len(job.times)
        self.dt = (job.times[-1] - job.times[0]).astype(float)
        for typename in  job.schemas.keys():
            if typename in freq_list:
                self.pmc  = typename
                self.freq = freq_list[typename]
            if typename in imc_list:
                self.imc = typename
            if typename in cha_list:
                self.cha = typename
    
    def get_type(self, typename, aggregate = True):
        if typename == "imc": typename = self.imc
        if typename == "pmc": typename = self.pmc
        if typename == "cha": typename = self.cha
        if not typename: return
    
        schema = self.job.schemas[typename]
        stats = {}
        for hostname, host in self.job.hosts.items():
            if aggregate:
                stats[hostname] = 0
                for devname in host.stats[typename]:
                    stats[hostname] += host.stats[typename][devname].astype(float)
            else:
                stats[hostname] = {}
                for devname in host.stats[typename]:
                    stats[hostname][devname] = host.stats[typename][devname].astype(float)
        return schema, stats

In [20]:
u_list = []
for job in job_objects:
    u_list.append( utils(job) )

In [21]:
u_list

[<__main__.utils instance at 0x2b77efeff8c0>,
 <__main__.utils instance at 0x2b77f00f1ea8>,
 <__main__.utils instance at 0x2b77f00f1368>]

### Schema titles

In [ ]:
def formatTitles ( df ):
    return df.rename( columns={"index": "Job Name",
              "amd64_pmc": "AMD Opteron performance counters (per core)",
              "intel_hsw": "Intel Haswell Processor (HSW) (per core)",
              "intel_hsw_ht": "Intel Haswell Processor - Hyper-threaded (per logical core)",
              "intel_nhm": "Intel Nehalem Processor (NHM) (per core)",
              "intel_uncore": "Westmere Uncore (WTM) (per socket)",
              "intel_snb": "Intel Sandy Brige (SNB) or Ivy Bridge (IVB) Processor (per core)",
              "intel_rapl": "Running average power limit",
              "intel_hsw_cbo": "Caching Agent (CBo) for SNB (HSW) (per socket)",
              "intel_hsw_pcu": "Power Control Unit for SNB (HSW) (per socket)",
              "intel_hsw_imc": "Integrated Memory Controller for SNB (HSW) (per socket)",
              "intel_hsw_qpi": "QPI Link Layer for SNB (HSW) (per socket)",
              "intel_hsw_hau": "Home Agent Unit for SNB (HSW) (per socket)",
              "intel_hsw_r2pci": "Ring to PCIe Agent for SNB (HSW) (per socket)",
              "ib": "Infiniband usage (default)",
              "ib_sw": "InfiniBand usage (sw)",
              "ib_ext": "Infiniband usage (ext)",
              "llite": "Lustre filesystem usage (per mount)",
              "lnet": "Lustre network usage (lnet)",
              "mdc": "Lustre network usage (mdc)",
              "mic": "MIC scheduler account (per hardware thread)",
              "osc": "Lustre filesystem usage (osc)",
              "block": "Block device statistics (per device)",
              "cpu": "Scheduler accounting (per CPU)",
              "mem": "Memory usage (per socket)",
              "net": "Network device usage (per device)",
              "nfs": "NFS system usage",
              "numa": "NUMA statistics (per socket)",
              "proc": "Process specific data (MaxRSS, executable name etc.)",
              "ps": "Process statistics",
              "sysv_shm": "SysV shared memory segment usage",
              "tmpfs": "Ram-backed filesystem usage (per mount)",
              "vfs": "Dentry/file/inode cache usage",
              "vm": "Virtual memory statistics"
                            })

### Pre-Determined Metrics

In [56]:
def avg_blockbw(u):
    schema, _stats = u.get_type("block")
    blockbw = 0
    for hostname, stats in _stats.items():
        blockbw += stats[-1, schema["rd_sectors"].index] - stats[0, schema["rd_sectors"].index] + \
               stats[-1, schema["wr_sectors"].index] - stats[0, schema["wr_sectors"].index]
    return blockbw/(u.dt*u.nhosts*1024*1024)

def avg_cpi(u):
    schema, _stats = u.get_type("pmc")
    cycles = 0
    instrs = 0
    for hostname, stats in _stats.items():
        cycles += stats[-1, schema["CLOCKS_UNHALTED_CORE"].index] - \
            stats[0, schema["CLOCKS_UNHALTED_CORE"].index]
        instrs += stats[-1, schema["INSTRUCTIONS_RETIRED"].index] - \
            stats[0, schema["INSTRUCTIONS_RETIRED"].index] 
    return cycles/instrs

def avg_freq(u):
    schema, _stats = u.get_type("pmc")
    cycles = 0
    cycles_ref = 0
    for hostname, stats in _stats.items():
        cycles += stats[-1, schema["CLOCKS_UNHALTED_CORE"].index] - \
            stats[0, schema["CLOCKS_UNHALTED_CORE"].index]
        cycles_ref += stats[-1, schema["CLOCKS_UNHALTED_REF"].index] - \
                stats[0, schema["CLOCKS_UNHALTED_REF"].index] 
    return u.freq*cycles/cycles_ref

def avg_cpuusage(u):
    schema, _stats = u.get_type("cpu")    
    cpu = 0
    for hostname, stats in _stats.items():
        cpu += stats[-1, schema["user"].index] - stats[0, schema["user"].index]
    return cpu/(u.dt*u.nhosts*100)

def avg_ethbw(u):
    schema, _stats = u.get_type("net")
    bw = 0
    for hostname, stats in _stats.items():
        bw += stats[-1, schema["rx_bytes"].index] - stats[0, schema["rx_bytes"].index] + \
              stats[-1, schema["tx_bytes"].index] - stats[0, schema["tx_bytes"].index]
    return bw/(u.dt*u.nhosts*1024*1024)

def avg_fabricbw(u):
    avg = 0
    try:
        schema, _stats = u.get_type("ib_ext")              
        tb, rb = schema["port_xmit_data"].index, schema["port_rcv_data"].index
        conv2mb = 1024*1024
    except:
        schema, _stats = u.get_type("opa")  
        tb, rb = schema["PortXmitData"].index, schema["PortRcvData"].index
        conv2mb = 125000
    for hostname, stats in _stats.items():
        avg += stats[-1, tb] + stats[-1, rb] - \
               stats[0, tb] - stats[0, rb]
    return avg/(u.dt*u.nhosts*conv2mb)

def avg_flops(u):
    schema, _stats = u.get_type("pmc")
    vector_widths = {"SSE_D_ALL" : 1, "SIMD_D_256" : 2, 
                "FP_ARITH_INST_RETIRED_SCALAR_DOUBLE" : 1, 
                 "FP_ARITH_INST_RETIRED_128B_PACKED_DOUBLE" : 2, 
                 "FP_ARITH_INST_RETIRED_256B_PACKED_DOUBLE" : 4, 
                 "FP_ARITH_INST_RETIRED_512B_PACKED_DOUBLE" : 8, 
                 "SSE_DOUBLE_SCALAR" : 1, 
                 "SSE_DOUBLE_PACKED" : 2, 
                 "SIMD_DOUBLE_256" : 4}
    flops = 0
    for hostname, stats in _stats.items():
        for eventname in schema:
            if eventname in vector_widths:
                index = schema[eventname].index
                flops += (stats[-1, index] - stats[0, index])*vector_widths[eventname]
    return flops/(u.dt*u.nhosts*1e9)

def avg_l1loadhits(u):
    schema, _stats = u.get_type("pmc")
    load_names = ['LOAD_OPS_L1_HIT', 'MEM_UOPS_RETIRED_L1_HIT_LOADS']
    loads = 0
    for hostname, stats in _stats.items():
        for eventname in schema:
            if eventname in load_names:
                index = schema[eventname].index
                loads += stats[-1, index] - stats[0, index]
    return loads/(u.dt*u.nhosts)

def avg_l2loadhits(u):
    schema, _stats = u.get_type("pmc")
    load_names = ['LOAD_OPS_L2_HIT', 'MEM_UOPS_RETIRED_L2_HIT_LOADS']
    loads = 0
    for hostname, stats in _stats.items():
        for eventname in schema:
            if eventname in load_names:
                index = schema[eventname].index
                loads += stats[-1, index] - stats[0, index]
    return loads/(u.dt*u.nhosts)

def avg_llcloadhits(u):
    schema, _stats = u.get_type("pmc")
    load_names = ['LOAD_OPS_LLC_HIT', 'MEM_UOPS_RETIRED_LLC_HIT_LOADS']
    loads = 0
    for hostname, stats in _stats.items():
        for eventname in schema:
            if eventname in load_names:
                index = schema[eventname].index
                loads += stats[-1, index] - stats[0, index]
    return loads/(u.dt*u.nhosts)

def avg_lnetbw(u):
    schema, _stats = u.get_type("lnet")
    bw = 0
    for hostname, stats in _stats.items():
        bw += stats[-1, schema["rx_bytes"].index] + stats[-1, schema["tx_bytes"].index] \
              - stats[0, schema["rx_bytes"].index] - stats[0, schema["tx_bytes"].index]
    return bw/(1024*1024*u.dt*u.nhosts)

def avg_lnetmsgs(u):
    avg = 0
    schema, _stats = u.get_type("lnet")                  
    tx, rx = schema["tx_msgs"].index, schema["rx_msgs"].index

    for hostname, stats in _stats.items():
        avg += stats[-1, tx] + stats[-1, rx] - \
               stats[0, tx] - stats[0, rx]
    return avg/(u.dt*u.nhosts)

def avg_loads(u):
    schema, _stats = u.get_type("pmc")
    load_names = ['LOAD_OPS_ALL','MEM_UOPS_RETIRED_ALL_LOADS']
    loads = 0
    for hostname, stats in _stats.items():
        for eventname in schema:
            if eventname in load_names:
                index = schema[eventname].index
                loads += stats[-1, index] - stats[0, index]
    return loads/(u.dt*u.nhosts)

def avg_mbw(u):
    schema, _stats = u.get_type("imc")
    avg = 0
    for hostname, stats in _stats.items():
        avg += stats[-1, schema["CAS_READS"].index] + stats[-1, schema["CAS_WRITES"].index] \
             - stats[0, schema["CAS_READS"].index] - stats[0, schema["CAS_WRITES"].index]
    return 64.0*avg/(1024*1024*1024*u.dt*u.nhosts)

def avg_mcdrambw(u):      
    avg = 0
    schema, _stats = u.get_type("intel_knl_edc_eclk")
    for hostname, stats in _stats.items():
        avg += stats[-1, schema["RPQ_INSERTS"].index] + stats[-1, schema["WPQ_INSERTS"].index] \
             - stats[0, schema["RPQ_INSERTS"].index] - stats[0, schema["WPQ_INSERTS"].index]

    if not "flat" in u.job.acct["queue"].lower():
        schema, _stats = u.get_type("intel_knl_edc_uclk")
        for hostname, stats in _stats.items():
            avg -= stats[-1, schema["EDC_MISS_CLEAN"].index] - stats[0, schema["EDC_MISS_CLEAN"].index] + \
                stats[-1, schema["EDC_MISS_DIRTY"].index] - stats[0, schema["EDC_MISS_DIRTY"].index]

        schema, _stats = u.get_type("intel_knl_mc_dclk")
        for hostname, stats in _stats.items():
            avg -= stats[-1, schema["CAS_READS"].index] - stats[0, schema["CAS_READS"].index]

    return 64.0*avg/(1024*1024*1024*u.dt*u.nhosts)

def avg_mdcreqs(u):
    schema, _stats = u.get_type("mdc")
    idx = schema["reqs"].index
    avg = 0
    for hostname, stats in _stats.items():
        avg += stats[-1, idx] - stats[0, idx]
    return avg/(u.dt*u.nhosts)

def avg_mdcwait(u):
    schema, _stats = u.get_type("mdc")
    idx0, idx1 = schema["reqs"].index, schema["wait"].index
    avg0, avg1 = 0, 0 
    for hostname, stats in _stats.items():
        avg0 += stats[-1, idx0] - stats[0, idx0]
        avg1 += stats[-1, idx1] - stats[0, idx1]
    return avg1/avg0

def avg_openclose(u):
    schema, _stats = u.get_type("llite")
    idx0, idx1 = schema["open"].index, schema["close"].index
    avg = 0
    for hostname, stats in _stats.items():
        avg += stats[-1, idx0] - stats[0, idx0] + \
            stats[-1, idx1] - stats[0, idx1]
    return avg/(u.dt*u.nhosts)

def avg_oscreqs(u):
    schema, _stats = u.get_type("osc")
    idx = schema["reqs"].index
    avg = 0
    for hostname, stats in _stats.items():
        avg += stats[-1, idx] - stats[0, idx]
    return avg/(u.dt*u.nhosts)

def avg_oscwait(u):
    schema, _stats = u.get_type("osc")
    idx0, idx1 = schema["reqs"].index, schema["wait"].index
    avg0, avg1 = 0, 0 
    for hostname, stats in _stats.items():
        avg0 += stats[-1, idx0] - stats[0, idx0]
        avg1 += stats[-1, idx1] - stats[0, idx1]
    return avg1/avg0

def avg_packetsize(u):
    try:
        schema, _stats = u.get_type("ib_ext")              
        tx, rx = schema["port_xmit_pkts"].index, schema["port_rcv_pkts"].index
        tb, rb = schema["port_xmit_data"].index, schema["port_rcv_data"].index
        conv2mb = 1024*1024
    except:
        schema, _stats = u.get_type("opa")  
        tx, rx = schema["PortXmitPkts"].index, schema["PortRcvPkts"].index
        tb, rb = schema["PortXmitData"].index, schema["PortRcvData"].index
        conv2mb = 125000

    npacks = 0
    nbytes  = 0
    for hostname, stats in _stats.items():
        npacks += stats[-1, tx] + stats[-1, rx] - \
            stats[0, tx] - stats[0, rx]
        nbytes += stats[-1, tb] + stats[-1, rb] - \
            stats[0, tb] - stats[0, rb]
    return nbytes/(npacks*conv2mb)

def max_fabricbw(u):
    max_bw=0
    try:
        schema, _stats = u.get_type("ib_ext")              
        tx, rx = schema["port_xmit_data"].index, schema["port_rcv_data"].index
        conv2mb = 1024*1024
    except:
        schema, _stats = u.get_type("opa")  
        tx, rx = schema["PortXmitData"].index, schema["PortRcvData"].index
        conv2mb = 125000
    for hostname, stats in _stats.items():
        max_bw = max(max_bw, amax(diff(stats[:, tx] + stats[:, rx])/diff(u.t)))
    return max_bw/conv2mb

def max_lnetbw(u):
    max_bw=0.0
    schema, _stats = u.get_type("lnet")              
    tx, rx = schema["tx_bytes"].index, schema["rx_bytes"].index
    for hostname, stats in _stats.items():
        max_bw = max(max_bw, amax(diff(stats[:, tx] + stats[:, rx])/diff(u.t)))
    return max_bw/(1024*1024)

def max_mds(u):
    max_mds = 0
    schema, _stats = u.get_type("llite")  
    for hostname, stats in _stats.items():
        max_mds = max(max_mds, amax(diff(stats[:, schema["open"].index] + \
                                   stats[:, schema["close"].index] + \
                                   stats[:, schema["mmap"].index] + \
                                   stats[:, schema["fsync"].index] + \
                                   stats[:, schema["setattr"].index] + \
                                   stats[:, schema["truncate"].index] + \
                                   stats[:, schema["flock"].index] + \
                                   stats[:, schema["getattr"].index] + \
                                   stats[:, schema["statfs"].index] + \
                                   stats[:, schema["alloc_inode"].index] + \
                                   stats[:, schema["setxattr"].index] + \
                                   stats[:, schema["listxattr"].index] + \
                                   stats[:, schema["removexattr"].index] + \
                                   stats[:, schema["readdir"].index] + \
                                   stats[:, schema["create"].index] + \
                                   stats[:, schema["lookup"].index] + \
                                   stats[:, schema["link"].index] + \
                                   stats[:, schema["unlink"].index] + \
                                   stats[:, schema["symlink"].index] + \
                                   stats[:, schema["mkdir"].index] + \
                                   stats[:, schema["rmdir"].index] + \
                                   stats[:, schema["mknod"].index] + \
                                   stats[:, schema["rename"].index])/diff(u.t)))
    return max_mds

def max_packetrate(u):
    max_pr=0
    try:
        schema, _stats = u.get_type("ib_ext")              
        tx, rx = schema["port_xmit_pkts"].index, schema["port_rcv_pkts"].index
    except:
        schema, _stats = u.get_type("opa")  
        tx, rx = schema["PortXmitPkts"].index, schema["PortRcvPkts"].index

    for hostname, stats in _stats.items():
        max_pr = max(max_pr, amax(diff(stats[:, tx] + stats[:, rx])/diff(u.t)))
    return max_pr

# This will compute the maximum memory usage recorded
# by monitor.  It only samples at x mn intervals and
# may miss high water marks in between.   
def mem_hwm(u):
    # mem usage in GB
    max_memusage = 0.0 
    schema, _stats = u.get_type("mem")
    for hostname, stats in _stats.items():
        max_memusage = max(max_memusage, amax(stats[:, schema["MemUsed"].index] - \
                          stats[:, schema["Slab"].index] - \
                          stats[:, schema["FilePages"].index]))
    return max_memusage/(2.**30)

def node_imbalance(u):
    schema, _stats = u.get_type("cpu")
    max_usage = zeros(u.nt - 1)
    for hostname, stats in _stats.items():
        max_usage = maximum(max_usage, diff(stats[:, schema["user"].index])/diff(u.t))

    max_imbalance = []
    for hostname, stats in _stats.items():
        max_imbalance += [mean((max_usage - diff(stats[:, schema["user"].index])/diff(u.t))/max_usage)]    
    return amax([0. if isnan(x) else x for x in max_imbalance])

def time_imbalance(u):
    tmid=(u.t[:-1] + u.t[1:])/2.0
    dt = diff(u.t)
    schema, _stats = u.get_type("cpu")    
    vals = []
    for hostname, stats in _stats.items():
        #skip first and last two time slices
        for i in [x + 2 for x in range(len(u.t) - 4)]:
            r1=range(i)
            r2=[x + i for x in range(len(dt) - i)]
            rate = diff(stats[:, schema["user"].index])/diff(u.t)
            # integral before time slice 
            a = trapz(rate[r1], tmid[r1])/(tmid[i] - tmid[0])
            # integral after time slice
            b = trapz(rate[r2], tmid[r2])/(tmid[-1] - tmid[i])
            # ratio of integral after time over before time
            vals += [b/a]        
    if vals:
        return min(vals)
    else:
        return None

def avg_sf_evictrate(u):
    schema, _stats = u.get_type("cha")
    sf_evictions = 0
    llc_lookup = 0                  
    for hostname, stats in _stats.items():
        sf_evictions += stats[-1, schema["SF_EVICTIONS_MES"].index] - \
                  stats[0, schema["SF_EVICTIONS_MES"].index]
        llc_lookup   += stats[-1, schema["LLC_LOOKUP_DATA_READ_LOCAL"].index] - \
                  stats[0, schema["LLC_LOOKUP_DATA_READ_LOCAL"].index] 
    return sf_evictions/llc_lookup

def avg_page_hitrate(u):
    schema, _stats = u.get_type("imc")
    act = 0
    cas = 0                  
    for hostname, stats in _stats.items():
        act += stats[-1, schema["ACT_COUNT"].index] - \
             stats[0, schema["ACT_COUNT"].index]
        cas += stats[-1, schema["CAS_READS"].index] + stats[-1, schema["CAS_WRITES"].index] - \
             stats[0, schema["CAS_READS"].index] - stats[0, schema["CAS_WRITES"].index]
    return (cas - act) / cas

def max_sf_evictrate(u):
    schema, _stats = u.get_type("cha", aggregate = False)
    max_rate = 0
    for hostname, dev in _stats.items():    
        sf_evictions = {}
        llc_lookup = {}
        
        for devname, stats in dev.items():
            socket = devname.split('/')[0]
            sf_evictions.setdefault(socket, 0)
            sf_evictions[socket] += stats[-1, schema["SF_EVICTIONS_MES"].index] - \
                                    stats[0, schema["SF_EVICTIONS_MES"].index]
            llc_lookup.setdefault(socket, 0)
            llc_lookup[socket]   += stats[-1, schema["LLC_LOOKUP_DATA_READ_LOCAL"].index] - \
                                    stats[0, schema["LLC_LOOKUP_DATA_READ_LOCAL"].index]

    for socket in set([x.split('/')[0] for x in dev.keys()]):
        max_rate = max(sf_evictions[socket]/llc_lookup[socket], max_rate)
    return max_rate

def max_load15(u):
    max_load15 = 0.0 
    schema, _stats = u.get_type("ps")
    for hostname, stats in _stats.items():
        max_load15 = max(max_load15, amax(stats[:, schema["load_15"].index]))
    return max_load15/100

In [58]:
metrics = {
    "avg_cpuusage [#cores]": avg_cpuusage( u_list[0] ),
    "mem_hwm [GB]": mem_hwm ( u_list[0] ),
    "node_imbalance":  node_imbalance( u_list[0] ),
    "time_imbalance":  time_imbalance( u_list[0] ),
    "avg_flops [GF]":  avg_flops( u_list[0] ),
    "avg_cpi [cyc/ins]":  avg_cpi( u_list[0] ),
    "avg_freq [GHz]":  avg_freq( u_list[0] ),
    "avg_loads [#/s]":  avg_loads ( u_list[0] ),
    "avg_l1loadhits [#/s]":  avg_l1loadhits( u_list[0] ),
    "avg_l2loadhits [#/s]":  avg_l2loadhits( u_list[0] ),
    "avg_llcloadhits [#/s]":  avg_llcloadhits ( u_list[0] ),
    "avg_sf_evictrate [#evicts/#rds]":  avg_sf_evictrate( u_list[0] ),
    "max_sf_evictrate [#evicts/#rds]":  max_sf_evictrate( u_list[0] ),
    "avg_mbw [GB/s]": avg_mbw( u_list[0] ),
    "avg_page_hitrate [hits/cas]": avg_page_hitrate( u_list[0] ),
    "avg_mcdrambw [GB/s]": avg_mcdrambw( u_list[0] ),
    "avg_fabricbw [MB/s]": avg_fabricbw( u_list[0] ),
    "max_fabricbw [MB/s]": max_fabricbw( u_list[0] ),
    "avg_packetsize [MB]": avg_packetsize( u_list[0] ),
    "max_packetrate [#/s]": max_packetrate( u_list[0] ),
    "avg_ethbw [MB/s]":  avg_ethbw  ( u_list[0] ),
    "max_mds [#/s]" : max_mds( u_list[0] ),
    "avg_lnetmsgs [#/s]": avg_lnetmsgs( u_list[0] ),
    "avg_lnetbw [MB/s]":  avg_lnetbw( u_list[0] ),
    "max_lnetbw [MB/s]":  max_lnetbw( u_list[0] ),
    "avg_mdcreqs [#/s]":  avg_mdcreqs( u_list[0] ),
    "avg_mdcwait [us]":  avg_mdcwait( u_list[0] ),
    "avg_oscreqs [#/s]":  avg_oscreqs( u_list[0] ),
    "avg_oscwait [us]":  avg_oscwait( u_list[0] ),
    "avg_openclose [#/s]": avg_openclose( u_list[0] ),
    "avg_blockbw [MB/s]": avg_blockbw( u_list[0] ),
    "max_load15 [cores]": max_load15( u_list[0] )
        }

AttributeError: utils instance has no attribute 'cha'